In [3]:
# -*- coding: utf-8 -*-
"""
НАСТОЛЬНЫЙ КАЛЬКУЛЯТОР С СОХРАНЕНИЕМ ИСТОРИИ
Версия на основе предоставленного примера
ДЗ: Создание настольного калькулятора с возможностью вывода расчетов в .txt файл
"""

import tkinter as tk
from datetime import datetime
import os

# Имя файла для сохранения истории вычислений
HISTORY_FILE = "calculations_history.txt"


class Calculator:
    """Класс калькулятора с графическим интерфейсом на Tkinter"""
    
    def __init__(self, root):
        """
        Инициализация калькулятора.
        
        Параметры:
            root: главное окно Tkinter
        """
        self.root = root
        # Настройка заголовка и размеров окна
        self.root.title("Калькулятор — ДЗ №7 с историей")
        self.root.geometry("300x450")  # Немного увеличил высоту для кнопки истории
        self.root.resizable(False, False)
        
        # Создаем файл истории, если он не существует
        self.create_history_file()
        
        # Строка для хранения текущего выражения
        self.expression = ""
        
        # === СОЗДАНИЕ ЭЛЕМЕНТОВ ИНТЕРФЕЙСА ===
        
        # === ДИСПЛЕЙ (поле для отображения ввода и результатов) ===
        self.display = tk.Entry(
            root,
            font=("Arial", 24),
            justify="right",  # Выравнивание по правому краю
            bd=10,           # Толщина рамки
            relief=tk.RIDGE  # Стиль рамки
        )
        # Размещаем дисплей в сетке (занимает все 4 колонки в первой строке)
        self.display.grid(row=0, column=0, columnspan=4, sticky="nsew", padx=5, pady=5)
        
        # === КНОПКИ КАЛЬКУЛЯТОРА ===
        # Формат: (текст_кнопки, строка, колонка)
        buttons = [
            # Первый ряд: 7, 8, 9, /
            ("7", 1, 0), ("8", 1, 1), ("9", 1, 2), ("/", 1, 3),
            # Второй ряд: 4, 5, 6, *
            ("4", 2, 0), ("5", 2, 1), ("6", 2, 2), ("*", 2, 3),
            # Третий ряд: 1, 2, 3, -
            ("1", 3, 0), ("2", 3, 1), ("3", 3, 2), ("-", 3, 3),
            # Четвертый ряд: 0, ., =, +
            ("0", 4, 0), (".", 4, 1), ("=", 4, 2), ("+", 4, 3),
            # Пятый ряд: C (очистка) и кнопка истории
            ("C", 5, 0), ("История", 5, 1, 2)  # История занимает 2 колонки
        ]
        
        # Создаем кнопки из списка
        for button_info in buttons:
            # Проверяем, занимает ли кнопка несколько колонок
            if len(button_info) == 4:  # Кнопка "История" занимает 2 колонки
                text, row, col, colspan = button_info
                # Создаем кнопку
                btn = tk.Button(
                    root,
                    text=text,
                    font=("Arial", 14),  # Чуть меньший шрифт для длинного текста
                    command=lambda v=text: self.on_button_click(v)
                )
                # Размещаем кнопку с указанием ширины (colspan)
                btn.grid(row=row, column=col, columnspan=colspan, 
                        sticky="nsew", padx=2, pady=2)
            else:
                text, row, col = button_info
                # Создаем кнопку
                btn = tk.Button(
                    root,
                    text=text,
                    font=("Arial", 18),
                    command=lambda v=text: self.on_button_click(v)
                )
                # Размещаем кнопку в сетке
                btn.grid(row=row, column=col, sticky="nsew", padx=2, pady=2)
        
        # === НАСТРОЙКА ЦВЕТОВ КНОПОК ===
        self.style_buttons()
        
        # === НАСТРОЙКА РАСШИРЕНИЯ СЕТКИ ===
        # Делаем строки и колонки растягиваемыми
        for i in range(6):  # 6 строк (0-5)
            root.grid_rowconfigure(i, weight=1)
        for i in range(4):  # 4 колонки (0-3)
            root.grid_columnconfigure(i, weight=1)
    
    def create_history_file(self):
        """Создает файл истории, если он не существует"""
        if not os.path.exists(HISTORY_FILE):
            try:
                # Создаем файл и записываем заголовок
                with open(HISTORY_FILE, "w", encoding="utf-8") as f:
                    f.write("История вычислений:\n")
                print(f"✅ Файл истории создан: {os.path.abspath(HISTORY_FILE)}")
            except Exception as e:
                print(f"❌ Ошибка создания файла: {e}")
        else:
            print(f"📁 Файл истории уже существует: {os.path.abspath(HISTORY_FILE)}")
    
    def style_buttons(self):
        """Настраивает цвета кнопок для лучшей визуализации"""
        # Получаем все дочерние элементы окна
        for child in self.root.winfo_children():
            if isinstance(child, tk.Button):
                text = child.cget("text")
                
                # Цвета для разных типов кнопок:
                if text in "0123456789.":
                    # Цифры и точка - светло-серый
                    child.config(bg="#e0e0e0", activebackground="#d0d0d0")
                elif text in "+-*/":
                    # Операторы - оранжевый
                    child.config(bg="#ff9800", fg="white", activebackground="#e68900")
                elif text == "=":
                    # Кнопка равно - зеленый
                    child.config(bg="#4CAF50", fg="white", activebackground="#45a049")
                elif text == "C":
                    # Кнопка очистки - красный
                    child.config(bg="#f44336", fg="white", activebackground="#d32f2f")
                elif text == "История":
                    # Кнопка истории - синий
                    child.config(bg="#2196F3", fg="white", activebackground="#1976D2")
    
    # === ОБРАБОТЧИКИ КНОПОК ===
    
    def on_button_click(self, value):
        """
        Обрабатывает нажатие кнопок калькулятора.
        
        Параметры:
            value: значение нажатой кнопки
        """
        # Обработка кнопки очистки
        if value == "C":
            self.expression = ""
            self.update_display()
            return
        
        # Обработка кнопки "История"
        if value == "История":
            self.show_history()
            return
        
        # Обработка кнопки равно
        if value == "=":
            self.calculate()
            return
        
        # Обработка кнопки точки (десятичный разделитель)
        if value == ".":
            # Проверяем, чтобы в текущем числе не было больше одной точки
            if self.expression and self.expression.split()[-1].count('.') < 1:
                self.expression += value
            elif not self.expression:
                # Если выражение пустое, начинаем с "0."
                self.expression = "0."
            self.update_display()
            return
        
        # Добавление цифр и операторов к выражению
        self.expression += value
        self.update_display()
    
    def update_display(self):
        """Обновляет содержимое дисплея"""
        self.display.delete(0, tk.END)  # Очищаем дисплей
        self.display.insert(0, self.expression)  # Вставляем текущее выражение
    
    # === ВЫЧИСЛЕНИЕ ===
    
    def calculate(self):
        """Выполняет вычисление выражения и сохраняет результат"""
        # Проверяем, есть ли выражение для вычисления
        if not self.expression:
            return
        
        try:
            # БЕЗОПАСНОЕ ВЫЧИСЛЕНИЕ ВЫРАЖЕНИЯ
            # Заменяем символы для корректной работы eval
            expr_to_eval = self.expression
            
            # Вычисляем результат
            result = str(eval(expr_to_eval))
            
            # Форматируем результат (убираем лишние нули после точки)
            if '.' in result:
                # Преобразуем к float для проверки
                result_float = float(result)
                if result_float.is_integer():
                    result = str(int(result_float))
                else:
                    # Убираем лишние нули в конце
                    result = result.rstrip('0').rstrip('.')
            
        except ZeroDivisionError:
            # Обработка деления на ноль
            result = "Ошибка: деление на 0"
        except Exception as e:
            # Обработка других ошибок
            result = "Ошибка"
            print(f"Ошибка вычисления: {e}")
        
        # === ВАЖНО: ЗАПИСЬ В ФАЙЛ ИСТОРИИ ===
        # Сохраняем ВСЕ операции, даже с ошибками
        self.log_to_file(self.expression, result)
        
        # Обновляем выражение результатом (для продолжения вычислений)
        self.expression = result
        self.update_display()
    
    # === ЛОГИРОВАНИЕ (СОХРАНЕНИЕ В ФАЙЛ) ===
    
    def log_to_file(self, expr, result):
        """
        Сохраняет операцию в файл истории.
        
        Параметры:
            expr: математическое выражение
            result: результат вычисления или сообщение об ошибке
        """
        # Получаем текущие дату и время
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        # Формируем строку для записи в формате: [дата время] выражение = результат
        line = f"[{timestamp}] {expr} = {result}\n"
        
        try:
            # Открываем файл в режиме добавления (append)
            # encoding="utf-8" для корректного отображения кириллицы
            with open(HISTORY_FILE, "a", encoding="utf-8") as f:
                f.write(line)
            print(f"✅ Сохранено в историю: {expr} = {result}")
        except Exception as e:
            print(f"❌ Ошибка записи в файл: {e}")
    
    def show_history(self):
        """Отображает историю вычислений в отдельном окне"""
        # Создаем новое окно для отображения истории
        history_window = tk.Toplevel(self.root)
        history_window.title("История вычислений")
        history_window.geometry("500x400")
        
        # Добавляем заголовок
        title_label = tk.Label(
            history_window,
            text="История вычислений",
            font=("Arial", 16, "bold")
        )
        title_label.pack(pady=10)
        
        # Добавляем информацию о файле
        file_label = tk.Label(
            history_window,
            text=f"Файл: {os.path.abspath(HISTORY_FILE)}",
            font=("Arial", 10),
            fg="gray"
        )
        file_label.pack()
        
        # Создаем текстовое поле с прокруткой для отображения истории
        text_frame = tk.Frame(history_window)
        text_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Добавляем вертикальную полосу прокрутки
        scrollbar = tk.Scrollbar(text_frame)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        # Создаем текстовое поле
        history_text = tk.Text(
            text_frame,
            wrap=tk.WORD,
            yscrollcommand=scrollbar.set,
            font=("Consolas", 10),
            bg="#f9f9f9"
        )
        history_text.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        # Связываем полосу прокрутки с текстовым полем
        scrollbar.config(command=history_text.yview)
        
        try:
            # Читаем содержимое файла истории
            if os.path.exists(HISTORY_FILE):
                with open(HISTORY_FILE, "r", encoding="utf-8") as f:
                    content = f.read()
                    # Вставляем содержимое в текстовое поле
                    history_text.insert(tk.END, content)
            else:
                history_text.insert(tk.END, "Файл истории не найден.")
            
            # Делаем текстовое поле доступным только для чтения
            history_text.config(state=tk.DISABLED)
            
        except Exception as e:
            history_text.insert(tk.END, f"Ошибка чтения файла: {e}")
        
        # Добавляем кнопку закрытия
        close_button = tk.Button(
            history_window,
            text="Закрыть",
            font=("Arial", 12),
            command=history_window.destroy
        )
        close_button.pack(pady=10)


# === ЗАПУСК ПРИЛОЖЕНИЯ ===
if __name__ == "__main__":
    # Создаем главное окно приложения
    root = tk.Tk()
    
    # Выводим информационное сообщение в консоль
    print("=" * 60)
    print("🚀 ЗАПУСК КАЛЬКУЛЯТОРА С СОХРАНЕНИЕМ ИСТОРИИ")
    print("=" * 60)
    print("📋 ТРЕБОВАНИЯ К ДЗ:")
    print("1. Арифметические операции: +, -, *, / ✓")
    print("2. Кнопки для чисел 0-9 ✓")
    print("3. Кнопка '=' для вычисления ✓")
    print("4. Кнопка 'C' для очистки ✓")
    print("5. Дисплей для отображения ввода и результатов ✓")
    print("6. Автосохранение ВСЕХ операций в файл ✓")
    print("7. Формат записи: [дата время] выражение = результат ✓")
    print("=" * 60)
    print("💡 ИНСТРУКЦИЯ:")
    print("• Нажимайте цифры для ввода чисел")
    print("• Используйте . для десятичных дробей")
    print("• Выбирайте операции: +, -, *, /")
    print("• Нажмите = для вычисления")
    print("• Нажмите C для очистки")
    print("• Нажмите 'История' для просмотра всех операций")
    print("=" * 60)
    
    # Создаем экземпляр калькулятора
    app = Calculator(root)
    
    # Запускаем главный цикл обработки событий
    root.mainloop()

🚀 ЗАПУСК КАЛЬКУЛЯТОРА С СОХРАНЕНИЕМ ИСТОРИИ
📋 ТРЕБОВАНИЯ К ДЗ:
1. Арифметические операции: +, -, *, / ✓
2. Кнопки для чисел 0-9 ✓
3. Кнопка '=' для вычисления ✓
4. Кнопка 'C' для очистки ✓
5. Дисплей для отображения ввода и результатов ✓
6. Автосохранение ВСЕХ операций в файл ✓
7. Формат записи: [дата время] выражение = результат ✓
💡 ИНСТРУКЦИЯ:
• Нажимайте цифры для ввода чисел
• Используйте . для десятичных дробей
• Выбирайте операции: +, -, *, /
• Нажмите = для вычисления
• Нажмите C для очистки
• Нажмите 'История' для просмотра всех операций
📁 Файл истории уже существует: C:\Users\Angel\calculations_history.txt
✅ Сохранено в историю: 3.0+2 = 5
✅ Сохранено в историю: 56*3 = 168
✅ Сохранено в историю: 9/3 = 3
